In [2]:
import cv2
import mediapipe as mp

# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# 얼굴 랜드마크를 제외한 전신 랜드마크 연결
POSE_CONNECTIONS_NO_FACE = [
    (11, 12), (12, 14), (14, 16), (16, 20), (20, 18), (18, 16), (11, 13), (13, 15), (15, 19), (19, 17), (17, 15), 
    (11, 23), (12, 24), (23, 24), (23, 25), (24, 26), (25, 27), (26, 28), (27, 29), (28, 30), (29, 31), (30, 32)
]

# 웹캠에서 영상 받아오기
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # 이미지를 RGB로 변환
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # 포즈 인식
    results = pose.process(image)

    # 이미지를 다시 BGR로 변환 (cv2에서 출력하기 위해)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.pose_landmarks:
        # 얼굴 랜드마크 제외한 랜드마크 그리기
        for idx, landmark in enumerate(results.pose_landmarks.landmark):
            if idx > 10:  # 0-10번 랜드마크는 얼굴에 해당하므로 제외
                h, w, _ = image.shape
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(image, (cx, cy), 5, (0, 255, 0), cv2.FILLED)
        
        # 얼굴 랜드마크 제외한 연결 그리기
        for connection in POSE_CONNECTIONS_NO_FACE:
            start_idx, end_idx = connection
            start_landmark = results.pose_landmarks.landmark[start_idx]
            end_landmark = results.pose_landmarks.landmark[end_idx]
            start_coords = (int(start_landmark.x * w), int(start_landmark.y * h))
            end_coords = (int(end_landmark.x * w), int(end_landmark.y * h))
            cv2.line(image, start_coords, end_coords, (0, 255, 0), 2)

    cv2.imshow('Pose Estimation', image)

    if cv2.waitKey(5) & 0xFF == 27:  # ESC 키를 누르면 종료
        break

cap.release()
cv2.destroyAllWindows()
